# Taxi Compass
by Alejandro Seif

# **Day 1: Data Collection**

## LTA Taxi Availability data API

* Import statement

In [19]:
import requests
import pandas as pd
import time
from datetime import datetime

### Sequential Taxi Data API Interaction

This Information updates every 50-60 seconds, so this service can be called at most 1x per minute

In [30]:
gov = 'https://api.data.gov.sg/v1'
taxi_url = '/transport/taxi-availability'

url = gov + taxi_url
response = requests.get(url).json()

response

{'type': 'FeatureCollection',
 'crs': {'type': 'link',
  'properties': {'href': 'http://spatialreference.org/ref/epsg/4326/ogcwkt/',
   'type': 'ogcwkt'}},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPoint',
    'coordinates': [[103.619061166667, 1.27906536666667],
     [103.62537, 1.27539],
     [103.62537, 1.2758],
     [103.63092, 1.32002],
     [103.64278, 1.32967],
     [103.65751, 1.31602],
     [103.665565833333, 1.30568033333333],
     [103.67227, 1.33002],
     [103.6782, 1.33033],
     [103.68365, 1.34837],
     [103.688734833333, 1.34106733333333],
     [103.69187, 1.34723],
     [103.69235, 1.34293],
     [103.69255, 1.34682],
     [103.69305405, 1.34205151666667],
     [103.69314, 1.31112],
     [103.694180766667, 1.34476625],
     [103.694837216667, 1.34708483333333],
     [103.69604995, 1.34574571666667],
     [103.696065, 1.34568116666667],
     [103.69641, 1.33583],
     [103.69642, 1.33],
     [103.69719, 1.34],
     [103.69811, 1.33],
     [103.69

In [31]:
timestamp_str = response['features'][0]['properties']['timestamp']
timestamp = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%S+08:00')
print(timestamp)

2021-12-16 19:47:18


In [47]:
taxi_coordinates = response['features'][0]['geometry']['coordinates']
taxi_coordinates_df= pd.DataFrame.from_dict(taxi_coordinates)
taxi_coordinates_df= taxi_coordinates_df.rename(columns = {0:'lon',1:'lat'})
taxi_coordinates_df['timestamp']=timestamp
taxi_coordinates_df.astype({'lat':'float32','lon':'float32'})
taxi_coordinates_df

,lon,lat,timestamp
0,103.619061,1.279065,2021-12-16 19:47:18
1,103.625370,1.275390,2021-12-16 19:47:18
2,103.625370,1.275800,2021-12-16 19:47:18
3,103.630920,1.320020,2021-12-16 19:47:18
4,103.642780,1.329670,2021-12-16 19:47:18
...,...,...,...
2525,103.988805,1.361094,2021-12-16 19:47:18
2526,103.988860,1.360980,2021-12-16 19:47:18
2527,103.988890,1.361210,2021-12-16 19:47:18
2528,103.989050,1.360000,2021-12-16 19:47:18


Now we are going to be setting up a function to retrieve and append so that it can be called

In [46]:
def taxi_coordinates_append(df):
    '''
    Pass an existing dataframe to append more data
    '''
    gov = 'https://api.data.gov.sg/v1'
    taxi_url = '/transport/taxi-availability'

    url = gov + taxi_url
    response = requests.get(url).json()
    
    timestamp_str = response['features'][0]['properties']['timestamp']
    timestamp = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%S+08:00')
    
    taxi_coordinates = response['features'][0]['geometry']['coordinates']
    temp_taxi_coordinates_df= pd.DataFrame.from_dict(taxi_coordinates)
    temp_taxi_coordinates_df= temp_taxi_coordinates_df.rename(columns = {0:'lon',1:'lat'})
    temp_taxi_coordinates_df['timestamp']=timestamp
    temp_taxi_coordinates_df.astype({'lat':'float32','lon':'float32'})
    return pd.concat([df,temp_taxi_coordinates_df])
    

In [49]:
taxi_coordinates_append(taxi_coordinates_df)

,lon,lat,timestamp
0,103.619061,1.279065,2021-12-16 19:47:18
1,103.625370,1.275390,2021-12-16 19:47:18
2,103.625370,1.275800,2021-12-16 19:47:18
3,103.630920,1.320020,2021-12-16 19:47:18
4,103.642780,1.329670,2021-12-16 19:47:18
...,...,...,...
3081,103.988460,1.360600,2021-12-16 20:13:49
3082,103.988900,1.360000,2021-12-16 20:13:49
3083,103.990070,1.360900,2021-12-16 20:13:49
3084,103.998130,1.382430,2021-12-16 20:13:49


### Weather data API Interaction

This service updates once every 5 minutes